In [137]:
import os, glob, time, re, json, random
import cv2
import matplotlib.pyplot as plt
import moviepy.editor as mp
from moviepy.video.io.ffmpeg_tools import ffmpeg_extract_subclip

class PreprocessDownstream:
    def __init__(self, videoPathL, outputPath, recordDownstreamPath="./recordDownstream.json", videoLength=2.4, logDownstreamPath="./logDownstream.txt",\
                labelPath=None, classL=None, divide=5, posThreshold=0.75, negThreshold=0.0):
        self.videoPathL = videoPathL
        self.outputPath = outputPath
        self.recordDownstreamPath = recordDownstreamPath
        self.videoLength = videoLength
        self.logDownstreamPath = logDownstreamPath
        self.getOverallInfo()
        self.getLabel(labelPath)
        self.classL = classL + ["others"]
        self.divide = divide
        self.posThreshold = posThreshold
        self.negThreshold = negThreshold
        self.getShifts()
        
    def getOverallInfo(self, rotateL=None):
        path = os.path.dirname(self.videoPathL[0])
        wantProcessedS = set(map(lambda path:path.split("/")[-1], self.videoPathL))
        hasProcessedS  = json.load(open(self.recordDownstreamPath,"r")).keys() if glob.glob(self.recordDownstreamPath) else set()
        needProcessedS = wantProcessedS - hasProcessedS
        print(f"len(hasProcessedS)={len(hasProcessedS)}")
        print(f"len(needProcessedS)={len(needProcessedS)}")
        self.videoPathL = sorted(list(filter(lambda path:path.split('/')[-1] in needProcessedS, self.videoPathL)))
        self.rotateL = rotateL if rotateL else [0]*len(needProcessedS)
        for i,videoPath in enumerate(self.videoPathL):
            frames, fps, height, width = self.getVideoInfo(videoPath)
            print(f"{i}, {videoPath.split('/')[-1]}, frames={frames}, fps={fps}, height={height}, width={width}")
            #self.show1Frame(videoPath, self.rotateL[i])
    
    def getVideoInfo(self, videoPath):
        cap    = cv2.VideoCapture(videoPath)
        frames = cap.get(cv2.CAP_PROP_FRAME_COUNT)
        fps    = cap.get(cv2.CAP_PROP_FPS)
        height = cap.get(cv2.CAP_PROP_FRAME_HEIGHT)
        width  = cap.get(cv2.CAP_PROP_FRAME_WIDTH)
        cap.release()
        return int(frames), round(fps), int(height), int(width)
    
    def show1Frame(self, videoPath, rotate=0): # rotate: 1 right-90, 2 left-90, 5 vflip
        cap = cv2.VideoCapture(videoPath)
        success, img = cap.read()
        if success:
            if rotate==1:
                img = cv2.rotate(img,cv2.ROTATE_90_CLOCKWISE)
            elif rotate==2:
                img = cv2.rotate(img,cv2.ROTATE_90_COUNTERCLOCKWISE)
            elif rotate==5:
                img = cv2.rotate(img,cv2.ROTATE_180)
            plt.imshow(img[:,:,::-1])
            plt.show()
        cap.release()
        
    def getLabel(self, labelPath):
        self.labelD = {}
        for line in open(labelPath,"r").readlines():
            line = line.replace("\n","").replace(" ","")
            if "[" in line:
                videoName = line[1:-1]
                self.labelD[videoName] = {"direction":-1, "times":[]}
            else:
                for cid,timeSlot in enumerate( line.split(",") ):
                    if timeSlot in ['','-'*23]:
                        continue
                    start, end = timeSlot.split("-")
                    (sh,sm,ss), (eh,em,es) = start.split(":"), end.split(":")
                    start, end = int(sh)*3600+int(sm)*60+float(ss), int(eh)*3600+int(em)*60+float(es)
                    assert 2.5<=end-start<=4, timeSlot                    
                    self.labelD[videoName]['times'].append( (cid,start,end) )
        #assert len(self.videoPathL)==len(self.labelD), (len(self.videoPathL),len(self.labelD))
        print(f"self.labelD={self.labelD}")
    
    def getShifts(self):
        flags = [ round(self.videoLength/self.divide*i,2)  for i in range(self.divide+1) ] # e.g. self.videoLength=2.4, self.divide=5, flag=[0.0, 0.48, 0.96, 1.44, 1.92, 2.4]
        self.shifts = [0]+[round(flags[i]+(flags[i+1]-flags[i])*random.Random(i+7).random(),2) for i in range(self.divide)]
        print(f"self.shifts={self.shifts}")
    
    def getPositiveCuts(self, gtTuple):
        cid, gtStart, gtEnd = gtTuple
        closestIntStart = int(gtStart/self.videoLength)*self.videoLength
        positiveCuts = []
        for start in [ closestIntStart-self.videoLength, closestIntStart, closestIntStart+self.videoLength]:
            for shift in self.shifts:
                cutStart = round(start+shift,2)
                cutEnd = round(cutStart+self.videoLength,2)
                if 0 <= cutStart <= gtStart <= cutEnd and (cutEnd-gtStart)/self.videoLength>=self.posThreshold:
                    positiveCuts.append( (cid,cutStart,cutEnd) )
                elif 0 <= gtStart <= cutStart <= cutEnd <= gtEnd:
                    positiveCuts.append( (cid,cutStart,cutEnd) )
                elif 0 <= cutStart <= gtEnd <= cutEnd and (gtEnd-cutStart)/self.videoLength>=self.posThreshold:
                    positiveCuts.append( (cid,cutStart,cutEnd) )
        return positiveCuts

    def getNegativeCuts(self, gtTuple1, gtTuple2):
        (_, _, gt1End), (_, gt2Start, _) = gtTuple1, gtTuple2
        negativeCuts = []
        for i in range(-1,int(gt2Start-gt1End/self.videoLength)+1):
            start = gt1End + i*2.4
            for shift in self.shifts:
                cutStart = round(start+shift,2)
                cutEnd = round(cutStart+self.videoLength,2)
                if 0 <= cutStart < gt1End < cutEnd < gt2Start and (gt1End-cutStart)/self.videoLength<self.negThreshold:
                    negativeCuts.append( (len(self.classL)-1,cutStart,cutEnd) )
                elif 0 <= gt1End < cutStart < cutEnd < gt2Start:
                    negativeCuts.append( (len(self.classL)-1,cutStart,cutEnd) )
                elif 0 <= gt1End < cutStart < gt2Start < cutEnd and (cutEnd-gt2Start)/self.videoLength<self.negThreshold:
                    negativeCuts.append( (len(self.classL)-1,cutStart,cutEnd) )
        return negativeCuts
        
    def clip(self):
        for i,cl in enumerate(self.classL):
            os.makedirs(f"{self.outputPath}/{i}_{cl}", exist_ok=True)
        #
        for i,videoPath in enumerate(self.videoPathL):
            videoName = videoPath.split("/")[-1]
            print(f"\r{i+1}/{len(self.videoPathL)}, {videoName}", end="")
            clip = mp.VideoFileClip(videoPath)
            if self.labelD[videoName]['direction']==1: # right90
                clip = clip.rotate(270)
            elif self.labelD[videoName]['direction']==2: # left 90
                clip = clip.rotate(90)
            elif self.labelD[videoName]['direction']==5: # vflip
                clip = clip.rotate(180)
            clip = clip.resize( (568,320) ) # modify aspect ratio (w,h)
            #
            times = self.labelD[videoName]['times']
            for j in range(len(times)):
                positiveCuts = self.getPositiveCuts(times[j])
                negativeCuts = self.getNegativeCuts(times[j-1], times[j])
                random.shuffle( negativeCuts )
                negativeCuts = negativeCuts[:int(len(positiveCuts)/(len(self.classL)-1-1))]
                cuts = positiveCuts + negativeCuts
                #print(times[j], cuts)
                for cid,cutStart,cutEnd in cuts:
                    try:
                        subclip = clip.subclip(cutStart,cutStart+self.videoLength)
                        savePath = f"{self.outputPath}/{cid}_{self.classL[cid]}/" + videoName.replace('.mp4',f'_{cid}_{cutStart}.mp4')
                        subclip.write_videofile(savePath, verbose=False, logger=None)
                    except Exception as e:
                        with open(self.logDownstreamPath,"a") as f:
                            f.write(str(e)+"\n")
                            
    def getClassDistribution(self):
        self.classCountL = [0]*len(self.classL)
        for newVideoPath in glob.glob(f"{self.outputPath}/*/*.mp4"):
            cid, _ = newVideoPath.split("/")[-2].split("_")
            self.classCountL[int(cid)]+=1
        print(f"self.classCountL={self.classCountL}")
        
    def check(self):
        newVideoPathL = sorted(glob.glob(f"{self.outputPath}/*/*.mp4"))
        for i,newVideoPath in enumerate(newVideoPathL):
            print(f"\r{i}/{len(newVideoPathL)}", end="")
            cap = cv2.VideoCapture(newVideoPath)
            success, img = cap.read()
            assert bool(success), newVideoPath
            frames, fps, height, width = self.getVideoInfo(newVideoPath)
            assert abs(frames-1)<=fps*self.videoLength, (frames,newVideoPath)
            cap.release()
        print("\ncheck video and frames complete")
    
    def generate_csv(self):
        newVideoPathL = glob.glob(f"{self.outputPath}/*/*.mp4")
        random.shuffle(newVideoPathL)
        cut = int(len(newVideoPathL)*0.8)
        for dset,(start,end) in zip(["train","val","test"], [(None,cut),(cut,None),(cut,None)]):
            with open(f"{self.outputPath}/{dset}.csv","w") as f:
                for newVideoPath in newVideoPathL[start:end]:
                    f.write(f"{os.path.abspath(newVideoPath)} {newVideoPath.split('/')[-2].split('_')[0]}\n")
            
    def updateRecord(self):
        oriD = json.load(open(self.recordDownstreamPath,"r")) if glob.glob(self.recordDownstreamPath) else {}
        json.dump({**oriD,**self.labelD},open(self.recordDownstreamPath,"w"))

In [135]:
videoPathL = sorted(glob.glob("/home/jovyan/data-vol-2/HAR/C10/20220810/*.mp4"))[:10]
obj = PreprocessDownstream( videoPathL, "../_data/downstream_0810_10", labelPath="../_data/downstream_0810_10/label.txt", classL=["scan","open","tear","close"] )
#obj.getOverallInfo(rotateL=None)
#obj.clip()
obj.getClassDistribution()
#obj.check()
#obj.generate_csv()
#obj.updateRecord()

len(hasProcessedS)=10
len(needProcessedS)=0
self.labelD={'video_20220810080429.mp4': {'direction': -1, 'times': []}, 'video_20220810080929.mp4': {'direction': -1, 'times': []}, 'video_20220810081429.mp4': {'direction': -1, 'times': []}, 'video_20220810081929.mp4': {'direction': -1, 'times': []}, 'video_20220810082429.mp4': {'direction': -1, 'times': [(0, 251.0, 253.5), (1, 258.0, 261.0), (2, 265.5, 268.0), (3, 293.5, 296.5)]}, 'video_20220810082929.mp4': {'direction': -1, 'times': [(0, 267.75, 270.25), (1, 274.0, 277.0), (2, 265.5, 268.0), (3, 281.0, 284.0)]}, 'video_20220810083429.mp4': {'direction': -1, 'times': [(3, 6.0, 9.0), (0, 49.75, 52.25), (1, 55.0, 58.0), (2, 64.0, 67.0), (3, 85.0, 88.0), (0, 108.75, 111.25), (1, 112.0, 114.5), (2, 118.3, 121.3), (3, 143.0, 145.5), (0, 160.25, 162.75), (1, 165.0, 167.5), (2, 198.0, 200.5), (3, 226.5, 229.5), (0, 273.75, 276.25), (1, 279.0, 282.0), (2, 287.5, 290.0)]}, 'video_20220810083930.mp4': {'direction': -1, 'times': [(3, 13.0, 15.5), (0

In [143]:
videoPathL = sorted(glob.glob("/home/jovyan/data-vol-2/HAR/C10/20220810/*.mp4"))[:20]
obj = PreprocessDownstream( videoPathL, "../_data/downstream_0810_20", labelPath="../_data/downstream_0810_20/label.txt", classL=["scan","tear","close"] )
#obj.getOverallInfo(rotateL=None)
obj.clip()
obj.getClassDistribution()
obj.check()
#obj.generate_csv()
#obj.updateRecord()

len(hasProcessedS)=0
len(needProcessedS)=20
0, video_20220810080429.mp4, frames=8921, fps=30, height=1080, width=1920
1, video_20220810080929.mp4, frames=8921, fps=30, height=1080, width=1920
2, video_20220810081429.mp4, frames=8929, fps=30, height=1080, width=1920
3, video_20220810081929.mp4, frames=8925, fps=30, height=1080, width=1920
4, video_20220810082429.mp4, frames=8937, fps=30, height=1080, width=1920
5, video_20220810082929.mp4, frames=8927, fps=30, height=1080, width=1920
6, video_20220810083429.mp4, frames=8908, fps=30, height=1080, width=1920
7, video_20220810083930.mp4, frames=8914, fps=30, height=1080, width=1920
8, video_20220810084430.mp4, frames=8930, fps=30, height=1080, width=1920
9, video_20220810084930.mp4, frames=8935, fps=30, height=1080, width=1920
10, video_20220810085430.mp4, frames=8928, fps=30, height=1080, width=1920
11, video_20220810085930.mp4, frames=8926, fps=30, height=1080, width=1920
12, video_20220810090430.mp4, frames=8938, fps=30, height=1080, wi